<a href="https://colab.research.google.com/github/Polarbeargo/PySyft/blob/master/examples/tutorials/Part%209%20-%20Intro%20to%20Encrypted%20Programs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 9 - Intro to Encrypted Programs

Believe it or not, it is possible to compute with encrypted data. In other words, it's possible to run a program where **ALL of the variables** in the program are **encrypted**! 

In this tutorial, we're going to walk through very basic tools of encrypted computation. In particular, we're going to focus on one popular approach called Secure Multi-Party Computation. In this lesson, we'll learn how to build an encrypted calculator which can perform calculations on encrypted numbers.

Authors:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)
- Théo Ryffel - GitHub: [@LaRiffle](https://github.com/LaRiffle)

References: 
- Morten Dahl - [Blog](https://mortendahl.github.io) - Twitter: [@mortendahlcs](https://twitter.com/mortendahlcs)

In [1]:
!pip install syft

     |████████████████████████████████| 184kB 2.8MB/s 
     |████████████████████████████████| 450kB 44.2MB/s 
     |████████████████████████████████| 389kB 41.9MB/s 
     |████████████████████████████████| 1.4MB 43.3MB/s 
     |████████████████████████████████| 71kB 18.7MB/s 
     |████████████████████████████████| 204kB 45.8MB/s 
     |████████████████████████████████| 51kB 19.6MB/s 
     |████████████████████████████████| 276kB 38.8MB/s 
     |████████████████████████████████| 122kB 37.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Step 1: Encryption Using Secure Multi-Party Computation

SMPC is at first glance a rather strange form of "encryption". Instead of using a public/private key to encrypt a variable, each value is split into multiple `shares`, each of which operate like a private key. Typically, these `shares` will be distributed amongst 2 or more _owners_. Thus, in order to decrypt the variable, all owners must agree to allow the decryption. In essence, everyone has a private key.

### Encrypt()

So, let's say we wanted to "encrypt" a variable `x`, we could do so in the following way.

 > Encryption doesn't use floats or real numbers but happens in a mathematical space called [integer quotient ring](http://mathworld.wolfram.com/QuotientRing.html) which is basically the integers between `0` and `Q-1`, where `Q` is prime and "big enough" so that the space can contain all the numbers that we use in our experiments. In practice, given a value `x` integer, we do `x % Q` to fit in the ring. (That's why we avoid using number `x' > Q`).

In [0]:
Q = 1234567891011

In [0]:
x = 25

In [0]:
import random

def encrypt(x):
    share_a = random.randint(0,Q)
    share_b = random.randint(0,Q)
    share_c = (x - share_a - share_b) % Q
    return (share_a, share_b,  share_c)

In [5]:
encrypt(x)

(202684609561, 719282787575, 312600493900)

As you can see here, we have split our variable `x` into 3 different shares, which could be sent to 3 different owners.

### Decrypt()

If we wanted to decrypt these 3 shares, we could simply sum them together and take the modulus of the result (mod Q)

In [0]:
def decrypt(*shares):
    return sum(shares) % Q

In [0]:
a,b,c = encrypt(25)

In [8]:
decrypt(a, b, c)

25

Importantly, notice that if we try to decrypt with only two shares, the decryption does not work!

In [9]:
decrypt(a, b)

529708408191

Thus, we need all of the owners to participate in order to decrypt the value. It is in this way that the `shares` act like private keys, all of which must be present in order to decrypt a value.

# Step 2: Basic Arithmetic Using SMPC

However, the truly extraordinary property of Secure Multi-Party Computation is the ability to perform computation **while the variables are still encrypted**. Let's demonstrate simple addition below. 

In [0]:
x = encrypt(25)
y = encrypt(5)

In [0]:
def add(x, y):
    z = list()
    # the first worker adds their shares together
    z.append((x[0] + y[0]) % Q)
    
    # the second worker adds their shares together
    z.append((x[1] + y[1]) % Q)
    
    # the third worker adds their shares together
    z.append((x[2] + y[2]) % Q)
    
    return z

In [12]:
decrypt(*add(x,y))

30

### Success!!!

And there you have it! If each worker (separately) adds their shares together, then the resulting shares will decrypt to the correct value (25 + 5 == 30). 

As it turns out, SMPC protocols exist which can allow this encrypted computation for the following operations:
- addition (which we've just seen)
- multiplication
- comparison

and using these basic underlying primitives, we can perform arbitrary computation!!!

In the next section, we're going to learn how to use the PySyft library to perform these operations!

# Step 3: SMPC Using PySyft

In the previous sections, we outlined some basic intuitions around SMPC is supposed to work. However, in practice we don't want to have to hand-write all of the primitive operations ourselves when writing our encrypted programs. So, in this section we're going to walk through the basics of how to do encrypted computation using PySyft. In particular, we're going to focus on how to do the 3 primitives previously mentioned: addition, multiplication, and comparison.

First, we need to create a few Virtual Workers (which hopefully you're now familiar with given our previous tutorials). 

In [0]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
bill = sy.VirtualWorker(hook, id="bill")

### Basic Encryption/Decryption

Encryption is as simple as taking any PySyft tensor and calling .share(). Decryption is as simple as calling .get() on the shared variable

In [0]:
x = torch.tensor([25])

In [15]:
x

tensor([25])

In [0]:
encrypted_x = x.share(bob, alice, bill)

In [17]:
encrypted_x.get()

tensor([25])

### Introspecting the Encrypted Values

If we look closer at Bob, Alice, and Bill's workers, we can see the shares that get created!

In [18]:
bob._objects

{}

In [0]:
x = torch.tensor([25]).share(bob, alice, bill)

In [20]:
# Bob's share
bobs_share = list(bob._objects.values())[0]
bobs_share

tensor([3389931506002870778])

In [21]:
# Alice's share
alices_share = list(alice._objects.values())[0]
alices_share

tensor([381409205872762694])

In [22]:
# Bill's share
bills_share = list(bill._objects.values())[0]
bills_share

tensor([-3771340711875633447])

And if we wanted to, we could decrypt these values using the SAME approach we talked about earlier!!!

In [23]:
Q = x.child.field

(bobs_share + alices_share + bills_share) % Q

tensor([25])

As you can see, when we called `.share()` it simply split the value into 3 shares and sent one share to each of the parties!

# Encrypted Arithmetic

And now you see that we can perform arithmetic on the underlying values! The API is constructed so that we can simply perform arithmetic like we would normal PyTorch tensors.

In [0]:
x = torch.tensor([25]).share(bob,alice)
y = torch.tensor([5]).share(bob,alice)

In [25]:
z = x + y
z.get()

tensor([30])

In [26]:
z = x - y
z.get()

tensor([20])

# Encrypted Multiplication

For multiplication we need an additional party who is responsible for consistently generating random numbers (and not colluding with any of the other parties). We call this person a "crypto provider". For all intensive purposes, the crpyto provider is just an additional VirtualWorker, but it's important to adknowledge that the crypto provider is not an "owner" in that he/she doesn't own shares but is someone who needs to be trusted to not collude with any of the existing shareholders.

In [0]:
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")

In [0]:
x = torch.tensor([25]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([5]).share(bob,alice, crypto_provider=crypto_provider)

In [29]:
# multiplication

z = x * y
z.get()

tensor([125])

You can also do matrix multiplication

In [0]:
x = torch.tensor([[1, 2],[3,4]]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([[2, 0],[0,2]]).share(bob,alice, crypto_provider=crypto_provider)

In [31]:
# matrix multiplication

z = x.mm(y)
z.get()

tensor([[2, 4],
        [6, 8]])

# Encrypted comparison
It is also possible to private comparisons between private values. We rely here on the SecureNN protocol, the details of which can be found [here](https://eprint.iacr.org/2018/442.pdf). The result of the comparison is also a private shared tensor.

In [0]:
x = torch.tensor([25]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([5]).share(bob,alice, crypto_provider=crypto_provider)

In [33]:
z = x > y
z.get()

tensor([1])

In [34]:
z = x <= y
z.get()

tensor([0])

In [35]:
z = x == y
z.get()

tensor([0])

In [36]:
z = x == y + 20
z.get()

tensor([1])

You can also perform max operations

In [37]:
x = torch.tensor([2, 3, 4, 1]).share(bob,alice, crypto_provider=crypto_provider)
x.max().get()

tensor([4])

In [38]:
x = torch.tensor([[2, 3], [4, 1]]).share(bob,alice, crypto_provider=crypto_provider)
max_values, max_ids = x.max(dim=0)
max_values.get()

tensor([4, 3])

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft on GitHub

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft GitHub Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for GitHub issues marked "good first issue".

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)